<a href="https://colab.research.google.com/github/STRIDES-Codes/NL4Cell-Integrating-NLP-with-single-cell-data-analysis/blob/main/CellAttention_with_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from google.colab import auth
auth.authenticate_user()

In [24]:
%%capture
!pip install git+https://github.com/raina-kikani/transformers.git

In [25]:
from transformers import EncoderDecoderModel, RobertaTokenizer, RobertaConfig, RobertaModel
import torch
import numpy as np

In [26]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(torch.cuda.get_device_name())
else:
  device = torch.device('cpu')
  

Tesla T4


## Train

In [27]:
!pip install tokenizers

In [28]:
!gsutil cp gs://cytereader/preprocessed_cell_corpus_0.txt . 
!gsutil cp gs://cytereader/preprocessed_cell_corpus_1.txt . 

Copying gs://cytereader/preprocessed_cell_corpus_0.txt...
\ [1 files][267.0 MiB/267.0 MiB]                                                
Operation completed over 1 objects/267.0 MiB.                                    
Copying gs://cytereader/preprocessed_cell_corpus_1.txt...
\ [1 files][267.0 MiB/267.0 MiB]                                                
Operation completed over 1 objects/267.0 MiB.                                    


In [29]:
from tokenizers import BertWordPieceTokenizer

!mkdir cellAttention
wb_tokenizer = BertWordPieceTokenizer(clean_text=True,
                                      strip_accents=True, lowercase=False)

wb_tokenizer.train(['preprocessed_cell_corpus_0.txt'],
                   vocab_size=10000, min_frequency=2,
                   special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
wb_tokenizer.save_model(".")

mkdir: cannot create directory ‘cellAttention’: File exists


['./vocab.txt']

In [32]:
from transformers import RobertaConfig, BertTokenizer, RobertaModel, RobertaForMaskedLM, RobertaForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('CellAttention', max_len=64)
configuration = RobertaConfig(vocab_size=1000)


model = RobertaForMaskedLM(configuration)

HTTPError: ignored

In [1]:
model.num_parameters()

NameError: ignored

In [ ]:
cnt = 0
with open('small.txt', 'w') as file:
  for line in open('preprocessed_cell_corpus_0.txt'):
    file.write(line)
    cnt += 1
    if cnt == 100000:
      break

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="small.txt",
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./cellAttention",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
# %%time
trainer.train()

In [ ]:
trainer.save_model("./cellAttention")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./cellAttention",
    tokenizer="./cellAttention"
)

In [ ]:
fill_mask("CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15- CD31_PECAM1- CD8a- CD182_CXCR2[MASK] CD66ace- CD63- CD14- CD66b- CD62L_Lselectin- CD3+ CD27- CD86+ CD10- CD197_CCR7+ CD28- CD11c- CD33- CD161- CD45RO- CD24- CD38+ CD278_ICOS- CD32- CD152_CTLA4+ IgM+ CD184_CXCR4+ CD279_PD1- CD56+ CD16-")

In [ ]:
input = 'CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15-'
masked_input = 'CD45+ CD196_CCR6+ CD181_CXCR1[MASKED] HLA_DR- CD15-'

output = 'CD45+ CD196_CCR6+ CD181_CXCR1-/+ HLA_DR- CD15-'